In [4]:
%matplotlib inline  
import matplotlib.pyplot as plt
from pamtra2.libs.singleScattering import scattering
from pamtra2.libs.singleScattering import Mie, Rayleigh, Tmatrix
from pamtra2.libs.singleScattering import self_similar_rayleigh_gans as ssrg
from pamtra2.libs import refractiveIndex
import numpy as np
rad2deg = 180.0/np.pi

# Angles
t0 = np.pi*0.5 # theta must be within 0 pi
t1 = np.pi*0.5
#t1 = np.pi-t0
p0 = 0.0*np.pi
p1 = 0.3*np.pi
#p1 = np.pi+p0
print(p1, p1 // (2.*np.pi))
#p1 = p1 - 2.*np.pi*(p1//(2.*np.pi))
print(t0, t1, p0, p1)

SyntaxError: invalid syntax (self_similar_rayleigh_gans.py, line 176)

In [3]:
# One centimeter particle, S-band 2.8GHz, pure ice sphere 
f = 94e9
d = 0.01
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')

cost=1.0
R = Rayleigh.RayleighScatt(d,frequency=f,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
M = Mie.MieScatt(d*cost,frequency=f/cost,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)
T = Tmatrix.TmatrixScatt(d*cost,frequency=f/cost,refractive_index=n, theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)

from pytmatrix import tmatrix, scatter, radar
scatt = tmatrix.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(R.Cext, R.Csca, R.Cabs,R.Cbck)
print(M.Cext, M.Csca, M.Cabs,M.Cbck)
print(T.Cext, T.Csca, T.Cabs,T.Cbck)

print(" ")
print("RAY")
print(" ")
print(R.S2, complex(0.,0.))
print(complex(0.,0.), R.S1)
print(" ")
print(R.S2/(R.S1*np.cos(p1-p0)))
print(" ")
print("MIE")
print(M.S2,complex(0.,0.))
print(complex(0.,0.),M.S1)
print(" ")
print(M.S2/(M.S1*np.cos(p1-p0)))
print(" ")
print("TMM")
print(T.S[0,0], complex(0.,0.))
print(complex(0.,0.), T.S[1,1])
print(" ")
print(T.S[1,1]/(T.S[0,0]*np.cos(p1-p0)))
print(" ")
print(scatt.get_S())
print(" ")
print(T.S)
print(" ")
print(" ")


Cext                    Csca                    Cabs                   Cbck            [m**2]
0.34915390735 0.349151464999 2.44235120109e-06 0.523727197499
0.000186599730593 0.000178732245048 7.86748554476e-06 0.000919845031585
0.000186599728595 0.0001787322473233511 7.86748127188e-06 0.00091984499428
 
RAY
 
(0.204148817152+0.000382899914217j) 0j
0j (0.119995663995+0.000225062922681j)
 
(2.894427191-7.68589613425e-19j)
 
MIE
(0.0008117428976867167-0.0019117250355033585j) 0j
0j (-0.00256599634962685-0.0026620960657562595j)
 
(0.37411536588071287+0.879381551058582j)
 
TMM
(0.000816408322532-0.00191828844653j) 0j
0j (-0.00257045404474-0.00267015910779j)
 
(1.18353190011-2.78339940215j)
 
[[  8.16408323e-04 -1.91828845e-03j  -5.47252364e-20 -1.43156974e-19j]
 [  5.47252364e-20 +1.43156974e-19j  -2.57045404e-03 -2.67015911e-03j]]
 
[[  8.16408323e-04 -1.91828845e-03j  -5.47252364e-20 -1.43156974e-19j]
 [  5.47252364e-20 +1.43156974e-19j  -2.57045404e-03 -2.67015911e-03j]]
 
 


In [6]:
p1-p0

3.141592653589793

In [3]:
# One millimeter particle, W-band 94GHz, pure ice sphere 
f = 94e9
d = 0.001
n = refractiveIndex.n(frequencies=f, temperatures=263.15, substance='ice')
rayC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=d, frequencies=f, refractive_indices=n, model='TMM', theta_inc=t0, theta_sca=t1, phi_inc=p0, phi_sca=p1)

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)
from pytmatrix import tmatrix, scatter, radar
scatt = tmatrix.Scatterer(radius=0.5*d, wavelength=299792458./f, m=n, axis_ratio=1.0/1.0, thet0=t0*rad2deg, thet=t1*rad2deg, phi0=p0*rad2deg, phi=p1*rad2deg)
print([scatter.ext_xsect(scatt),scatter.sca_xsect(scatt),scatter.ext_xsect(scatt)-scatter.sca_xsect(scatt),radar.radar_xsect(scatt)])

Cext                    Csca                    Cabs                   Cbck            [m**2]
(1.0498967461984795e-06, 1.0474543949973939e-06, 2.4423512010854983e-09, 5.2372719749869694e-07)
(3.8169026989341472e-07, 3.7719995847397647e-07, 4.4903114194382853e-09, 3.0198335883905455e-07)
(3.8169025547199783e-07, 3.771999482202861e-07, 4.4903072517117332e-09, 3.0198337240757998e-07)
[3.8169025547199783e-07, 3.771999482202861e-07, 4.4903072517117332e-09, 3.0198337240757998e-07]


In [4]:
# One millimeter particle, S-band 2.8GHz, pure ice sphere 
f=2.8e9
n = refractiveIndex.n(frequencies=f,temperatures=263.15,substance='ice')
rayC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Ray')
mieC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='Mie')
TmaC = scattering(diameters=0.0001, frequencies=f, refractive_indices=n, model='TMM')

print('Cext                    Csca                    Cabs                   Cbck            [m**2]')
print(rayC)
print(mieC)
print(TmaC)

Cext                    Csca                    Cabs                   Cbck            [m**2]
(3.1490912297415749e-15, 8.2461663911100578e-19, 3.1482666131024637e-15, 4.1230831955550289e-19)
(3.1485646987497757e-15, 2.7487285968888451e-19, 3.1482898258900869e-15, 4.1230740899916799e-19)
(3.1485646291424952e-15, 2.7487285368539355e-19, 3.14828975628881e-15, 4.1230742079816574e-19)


In [5]:
scatt.get_S()
scatt.get_Z()

array([[  2.40310732e-08,   0.00000000e+00,   2.66798438e-24,
         -3.94430453e-31],
       [  0.00000000e+00,   2.40310732e-08,  -9.61242926e-15,
         -0.00000000e+00],
       [ -2.66798438e-24,  -9.61242926e-15,  -2.40310732e-08,
         -4.70197740e-38],
       [  3.94430453e-31,   0.00000000e+00,  -4.70197740e-38,
         -2.40310732e-08]])